# 01 - Data Ingestion & Metadata Extraction

This notebook ingests the lifecycle catalog, resolves file locations, extracts metadata and textual signals, and materializes a feature table for downstream modeling.

## How to use
1. Update `EXCEL_PATH` and `RAW_BASE_DIR` if the catalog or files live elsewhere.
2. Run the notebook end-to-end to create `artifacts/metadata_features.csv` (and `.parquet`).
3. Review the diagnostics cells to understand missing files, extraction coverage, and class balance.

In [ ]:
# Optional: ensure dependencies are installed before running the notebook.
# !pip install pandas openpyxl python-magic-bin python-docx PyPDF2 python-pptx extract-msg textract tqdm pyxlsb

In [ ]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Iterable, Tuple

import numpy as np
import pandas as pd

try:
    import magic  # type: ignore
except ImportError:
    magic = None

try:
    from docx import Document  # type: ignore
except ImportError:
    Document = None

try:
    from pptx import Presentation  # type: ignore
except ImportError:
    Presentation = None

try:
    from PyPDF2 import PdfReader  # type: ignore
except ImportError:
    PdfReader = None

try:
    import extract_msg  # type: ignore
except ImportError:
    extract_msg = None

try:
    import textract  # type: ignore
except ImportError:
    textract = None

try:
    from tqdm.auto import tqdm  # type: ignore
    TQDM_AVAILABLE = True
except ImportError:
    TQDM_AVAILABLE = False

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

In [ ]:
# ---------------------------------------------------------------------------
# Configuration
# ---------------------------------------------------------------------------
PROJECT_ROOT = Path("..").resolve()
EXCEL_PATH = PROJECT_ROOT / "assets" / "training_data.xlsx"
RAW_BASE_DIR = PROJECT_ROOT  # adjust if files live elsewhere
OUTPUT_DIR = Path.cwd() / "artifacts"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DATA_PATH = OUTPUT_DIR / "metadata_features.csv"

REQUIRED_COLUMNS = ["Original File Path", "File Path", "Business Capability"]
OPTIONAL_COLUMNS = ["Record Type", "Retention Code", "Notes"]

print(f"Excel catalog path: {EXCEL_PATH}")
print(f"File base directory: {RAW_BASE_DIR}")

In [ ]:
RENAME_MAP = {
    "Original File Path": "original_file_path",
    "File Path": "file_path",
    "Business Capability": "business_capability",
    "Record Type": "record_type",
    "Retention Code": "retention_code",
    "Notes": "notes_text",
}


def load_catalog(path: Path, required_columns: Iterable[str], optional_columns: Iterable[str]) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f"Catalog not found: {path}")
    df = pd.read_excel(path)
    missing = [col for col in required_columns if col not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns in catalog: {missing}")
    available_optional = [col for col in optional_columns if col in df.columns]
    use_columns = list(required_columns) + available_optional
    df = df[use_columns].copy()
    df = df.rename(columns=RENAME_MAP)
    for column in ["original_file_path", "file_path", "business_capability"]:
        df[column] = df[column].astype(str).str.strip()
    for column in ["record_type", "retention_code", "notes_text"]:
        if column not in df.columns:
            df[column] = ""
        df[column] = df[column].fillna("").astype(str).str.strip()
    return df


catalog_df = load_catalog(EXCEL_PATH, REQUIRED_COLUMNS, OPTIONAL_COLUMNS)
catalog_df.head()

In [ ]:
KEYWORD_CLEANER = re.compile(r"[^A-Za-z0-9]+")


def resolve_path(raw_value: str, base_dir: Path) -> Path:
    candidate = Path(raw_value)
    if not candidate.is_absolute():
        candidate = base_dir / candidate
    return candidate.resolve()


def to_keywords_from_path(path: Path) -> str:
    tokens = []
    for part in path.parts:
        cleaned = KEYWORD_CLEANER.sub(" ", part).strip().lower()
        if cleaned:
            tokens.append(cleaned)
    return " ".join(tokens)


def to_keywords_from_text(value: str) -> str:
    cleaned = KEYWORD_CLEANER.sub(" ", value).strip().lower()
    return " ".join(token for token in cleaned.split() if token)


def path_depth(path: Path) -> int:
    return len(path.parts)


def gather_file_stats(path: Path) -> tuple[int, float]:
    try:
        stat_result = path.stat()
        return stat_result.st_size, float(stat_result.st_mtime)
    except OSError:
        return 0, float("nan")


EXTENSION_FAMILY_MAP = {
    ".xls": "excel",
    ".xlsx": "excel",
    ".xlsm": "excel",
    ".xlsb": "excel",
    ".csv": "tabular",
    ".tsv": "tabular",
    ".txt": "text",
    ".log": "text",
    ".json": "json",
    ".xml": "markup",
    ".yaml": "markup",
    ".yml": "markup",
    ".ini": "config",
    ".cfg": "config",
    ".conf": "config",
    ".doc": "word",
    ".docx": "word",
    ".pdf": "pdf",
    ".ppt": "presentation",
    ".pptx": "presentation",
    ".msg": "outlook",
    ".html": "html",
    ".htm": "html",
    ".css": "code",
    ".js": "code",
    ".sql": "code",
    ".py": "code",
    ".sas": "sas",
    ".sas7bdat": "sas",
    ".ipynb": "notebook",
    ".jpg": "image",
    ".jpeg": "image",
    ".png": "image",
    ".gif": "image",
    ".bmp": "image",
    ".vsd": "visio",
    ".vsdx": "visio",
    ".twb": "tableau",
    ".twbx": "tableau",
}


def extension_family(suffix: str) -> str:
    return EXTENSION_FAMILY_MAP.get(suffix.lower(), "other")


def count_digits(value: str) -> int:
    return sum(char.isdigit() for char in value)


catalog_df["resolved_path"] = catalog_df["file_path"].apply(lambda value: resolve_path(value, RAW_BASE_DIR))
catalog_df["original_resolved_path"] = catalog_df["original_file_path"].apply(lambda value: resolve_path(value, RAW_BASE_DIR))
catalog_df["file_exists"] = catalog_df["resolved_path"].apply(Path.exists)
catalog_df["extension"] = catalog_df["resolved_path"].apply(lambda p: p.suffix.lower())
catalog_df["extension_family"] = catalog_df["extension"].apply(extension_family)
catalog_df["file_name"] = catalog_df["resolved_path"].apply(lambda p: p.name)
catalog_df["file_stem"] = catalog_df["resolved_path"].apply(lambda p: p.stem)
catalog_df["file_name_keywords"] = catalog_df["file_name"].map(to_keywords_from_text)
catalog_df["file_stem_keywords"] = catalog_df["file_stem"].map(to_keywords_from_text)
catalog_df["path_keywords"] = catalog_df["resolved_path"].apply(to_keywords_from_path)
catalog_df["original_path_keywords"] = catalog_df["original_resolved_path"].apply(to_keywords_from_path)
catalog_df["path_depth"] = catalog_df["resolved_path"].apply(path_depth)
catalog_df["original_path_depth"] = catalog_df["original_resolved_path"].apply(path_depth)
catalog_df["path_token_count"] = catalog_df["path_keywords"].str.split().map(len)
catalog_df["original_path_token_count"] = catalog_df["original_path_keywords"].str.split().map(len)
catalog_df["path_char_len"] = catalog_df["resolved_path"].astype(str).str.len()
catalog_df["original_path_char_len"] = catalog_df["original_resolved_path"].astype(str).str.len()
catalog_df["file_name_char_len"] = catalog_df["file_name"].str.len()
catalog_df["file_name_digit_count"] = catalog_df["file_name"].map(count_digits)

size_mtime = catalog_df["resolved_path"].apply(gather_file_stats)
catalog_df["file_size_bytes"] = [pair[0] for pair in size_mtime]
catalog_df["modified_time_epoch"] = [pair[1] for pair in size_mtime]
catalog_df["modified_time_iso"] = pd.to_datetime(catalog_df["modified_time_epoch"], unit="s", errors="coerce")
catalog_df["notes_word_count"] = catalog_df["notes_text"].str.split().map(len)
catalog_df.head()

In [ ]:
TEXT_EXTENSIONS = {
    ".txt", ".csv", ".tsv", ".json", ".xml", ".yaml", ".yml", ".ini", ".cfg", ".conf",
    ".md", ".rst", ".sql", ".py", ".java", ".js", ".html", ".htm", ".css", ".log", ".sas", ".psv"
}
EXCEL_EXTENSIONS = {".xls", ".xlsx", ".xlsm", ".xlsb"}
HTML_EXTENSIONS = {".html", ".htm"}
MAX_CHAR_LENGTH = 5000


def read_text_file(path: Path) -> str:
    for encoding in ("utf-8", "utf-16", "latin-1", "cp1252"):
        try:
            return path.read_text(encoding=encoding, errors="ignore")
        except Exception:
            continue
    return ""


def normalize_text(text: str) -> str:
    return re.sub(r"\s+", " ", text).strip()


def strip_html_tags(text: str) -> str:
    text = re.sub(r"<script.*?>.*?</script>", " ", text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r"<style.*?>.*?</style>", " ", text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r"<[^>]+>", " ", text)
    return normalize_text(text)


def extract_text_from_msg(path: Path) -> str:
    if extract_msg is None:
        return ""
    try:
        message = extract_msg.Message(str(path))
        sections = [message.subject or "", message.body or ""]
        attachments = getattr(message, "attachments", [])
        for attachment in attachments:
            filename = getattr(attachment, "longFilename", "") or getattr(attachment, "filename", "")
            if filename:
                sections.append(filename)
        return normalize_text(" ".join(section for section in sections if section))
    except Exception:
        return ""


def excel_to_text(path: Path, suffix: str) -> str:
    read_kwargs = dict(sheet_name=None, dtype=str, nrows=200)
    if suffix == ".xlsb":
        read_kwargs["engine"] = "pyxlsb"
    text_chunks = []
    try:
        sheets = pd.read_excel(path, **read_kwargs)
    except Exception:
        return ""
    for sheet_name, sheet_df in sheets.items():
        text_chunks.append(f"__sheet__: {sheet_name}")
        for _, row in sheet_df.iterrows():
            row_text = " ".join(str(value) for value in row if pd.notna(value))
            if row_text:
                text_chunks.append(row_text)
    return "\n".join(text_chunks)


def extract_text_from_file(path: Path) -> str:
    if not path.exists() or not path.is_file():
        return ""
    suffix = path.suffix.lower()
    text_content = ""
    try:
        if suffix in TEXT_EXTENSIONS:
            text_content = read_text_file(path)
            if suffix in HTML_EXTENSIONS:
                text_content = strip_html_tags(text_content)
        elif suffix in EXCEL_EXTENSIONS:
            text_content = excel_to_text(path, suffix)
        elif suffix == ".pdf" and PdfReader is not None:
            try:
                reader = PdfReader(str(path))
                texts = []
                total_len = 0
                for page in reader.pages:
                    try:
                        page_text = page.extract_text() or ""
                    except Exception:
                        page_text = ""
                    if page_text:
                        texts.append(page_text)
                        total_len += len(page_text)
                        if total_len >= MAX_CHAR_LENGTH:
                            break
                text_content = "\n".join(texts)
            except Exception:
                text_content = ""
        elif suffix == ".docx" and Document is not None:
            try:
                document = Document(path)
                paragraphs = [paragraph.text for paragraph in document.paragraphs if paragraph.text]
                text_content = "\n".join(paragraphs)
            except Exception:
                text_content = ""
        elif suffix == ".pptx" and Presentation is not None:
            try:
                presentation = Presentation(path)
                texts = []
                for slide in presentation.slides:
                    for shape in slide.shapes:
                        if hasattr(shape, "text") and shape.text:
                            texts.append(shape.text)
                text_content = "\n".join(texts)
            except Exception:
                text_content = ""
        elif suffix == ".msg":
            text_content = extract_text_from_msg(path)
        elif textract is not None and suffix in {".doc", ".ppt"}:
            try:
                text_content = textract.process(str(path)).decode("utf-8", errors="ignore")
            except Exception:
                text_content = ""
        else:
            if magic is not None:
                try:
                    mime_type = magic.from_file(str(path), mime=True)
                except Exception:
                    mime_type = None
                if mime_type and "text" in mime_type:
                    text_content = read_text_file(path)
            if not text_content and textract is not None and suffix not in {".dll", ""}:
                try:
                    text_content = textract.process(str(path)).decode("utf-8", errors="ignore")
                except Exception:
                    text_content = ""
            if not text_content:
                text_content = read_text_file(path)
    except Exception:
        text_content = ""
    text_content = normalize_text(text_content)
    if suffix in HTML_EXTENSIONS:
        text_content = strip_html_tags(text_content)
    return text_content[:MAX_CHAR_LENGTH]


if TQDM_AVAILABLE:
    tqdm.pandas(desc="Extracting file content")
    catalog_df["content_text"] = catalog_df["resolved_path"].progress_apply(extract_text_from_file)
else:
    catalog_df["content_text"] = catalog_df["resolved_path"].apply(extract_text_from_file)

catalog_df["content_char_len"] = catalog_df["content_text"].str.len().fillna(0)
catalog_df["content_word_count"] = catalog_df["content_text"].str.split().map(len).fillna(0)
catalog_df["has_content"] = (catalog_df["content_char_len"] > 0).astype(int)
catalog_df.head()

In [ ]:
feature_df = catalog_df.copy()

feature_df["resolved_path"] = feature_df["resolved_path"].astype(str)
feature_df["original_resolved_path"] = feature_df["original_resolved_path"].astype(str)
feature_df["extension"] = feature_df["extension"].fillna("").astype(str)
feature_df["extension_family"] = feature_df["extension_family"].fillna("").astype(str)
feature_df["record_type"] = feature_df["record_type"].fillna("").astype(str)
feature_df["retention_code"] = feature_df["retention_code"].fillna("").astype(str)
feature_df["notes_text"] = feature_df["notes_text"].fillna("").astype(str)
feature_df["file_name_keywords"] = feature_df["file_name_keywords"].fillna("").astype(str)
feature_df["file_stem_keywords"] = feature_df["file_stem_keywords"].fillna("").astype(str)
feature_df["path_keywords"] = feature_df["path_keywords"].fillna("").astype(str)
feature_df["original_path_keywords"] = feature_df["original_path_keywords"].fillna("").astype(str)

numeric_fields = [
    "file_exists",
    "file_size_bytes",
    "modified_time_epoch",
    "path_depth",
    "original_path_depth",
    "path_token_count",
    "original_path_token_count",
    "path_char_len",
    "original_path_char_len",
    "file_name_char_len",
    "file_name_digit_count",
    "content_char_len",
    "content_word_count",
    "notes_word_count",
    "has_content",
]
for column in numeric_fields:
    if column in feature_df:
        feature_df[column] = feature_df[column].fillna(0)

feature_df["file_exists"] = feature_df["file_exists"].astype(int)
feature_df["has_content"] = feature_df["has_content"].astype(int)

feature_df["modified_time_iso"] = pd.to_datetime(feature_df["modified_time_iso"], errors="coerce")
feature_df["modified_time_iso"] = feature_df["modified_time_iso"].dt.strftime("%Y-%m-%dT%H:%M:%S")
feature_df["modified_time_iso"] = feature_df["modified_time_iso"].replace("NaT", "")

feature_df.to_csv(OUTPUT_DATA_PATH, index=False)
print(f"Saved curated dataset to {OUTPUT_DATA_PATH}")

try:
    parquet_path = OUTPUT_DIR / "metadata_features.parquet"
    feature_df.to_parquet(parquet_path, index=False)
    print(f"(Optional) Saved Parquet dataset to {parquet_path}")
except Exception as parquet_error:
    print("Parquet export skipped:", parquet_error)

feature_df[
    [
        "business_capability",
        "extension",
        "extension_family",
        "file_exists",
        "file_size_bytes",
        "content_char_len",
        "notes_word_count",
    ]
].head()

In [ ]:
catalog_df["extension"].value_counts().head(20)

In [ ]:
catalog_df["file_exists"].value_counts()

In [ ]:
catalog_df["has_content"].value_counts()

In [ ]:
catalog_df["business_capability"].value_counts()